# 1. Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from colorama import Fore
import skimage.io as io
import os
import pandas as pd
import random
# torch 
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision.models as models
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import math
 



In [ ]:
%cd /content/drive/MyDrive/domian_adoptation


/content/drive/MyDrive/domian_adoptation


In [ ]:
from  mean_and_std import get_mean_and_std
from dataloader.dataloader import CustomImageDataset ,roteted_dataset
from visualization.data_plot import plot_pic

# 3. Datasets




In [ ]:
if not os.path.isfile('/content/ROD-synROD/ROD/rgb-washington/apple/apple_1/apple_1_1_101_crop.png'):
        print(Fore.RED, 'This might take several minuets')
        ! tar -xf     /content/drive/MyDrive/domian_adoptation/' ROD-synROD.tar'  -C /content
else:
         print(Fore.RED, 'untar file already exist')


 untar file already exist


# 2.1 ROD dataset


---








In [ ]:
with open("/content/ROD-synROD/ROD/wrgbd_40k-split_sync.txt") as f:
 
    lines = f.read() 
    imgs = lines.split('\n')
imgs.remove('')
print(imgs[0])

all_labels = [imgs[i].split('/')[0] for i in range(len(imgs))]
all_labels=list(set(all_labels))
labelmap ={}
for i in range(len(all_labels)):
   labelmap[all_labels[i]] =i
print('total number of labels: ',Fore.RED,len(labelmap) )
print(Fore.BLUE,labelmap)

rubber_eraser/rubber_eraser_4/rubber_eraser_4_1_136_***.png 38
total number of labels:   47
 {'cap': 0, 'instant_noodles': 1, 'keyboard': 2, 'food_jar': 3, 'food_can': 4, 'shampoo': 5, 'dry_battery': 6, 'bowl': 7, 'cereal_box': 8, 'notebook': 9, 'greens': 10, 'pitcher': 11, 'garlic': 12, 'potato': 13, 'banana': 14, 'sponge': 15, 'water_bottle': 16, 'soda_can': 17, 'binder': 18, 'cell_phone': 19, 'pliers': 20, 'lightbulb': 21, 'lemon': 22, 'food_cup': 23, 'apple': 24, 'bell_pepper': 25, 'mushroom': 26, 'food_bag': 27, 'toothbrush': 28, 'glue_stick': 29, 'coffee_mug': 30, 'comb': 31, 'stapler': 32, 'hand_towel': 33, 'toothpaste': 34, 'scissors': 35, 'rubber_eraser': 36, 'calculator': 37, 'food_box': 38, 'orange': 39, 'ball': 40, 'kleenex': 41, 'marker': 42, 'camera': 43, 'plate': 44, 'flashlight': 45, 'pear': 46}


In [ ]:


ROD_dataset = CustomImageDataset(dt= 'ROD',annotations_list = imgs,label_map=labelmap,img_dir ="/content/ROD-synROD/ROD/rgb-washington",
                                 depth_dir ="/content/ROD-synROD/ROD/surfnorm-washington",
                                transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(), ]),
                                 
                                transform_d = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(), ]),
                                    
       
)

ROD_dataloader = DataLoader(ROD_dataset, batch_size= 256,shuffle=True)


mean, std,mean_d,std_d = get_mean_and_std(ROD_dataloader)

print('mean',mean)
print('std',std)
print('mean_d',mean_d)
print('std_d',std_d)


rubber_eraser/rubber_eraser_4/rubber_eraser_4_1_136_***.png 38
total number of labels:   47
 {'bell_pepper': 0, 'rubber_eraser': 1, 'calculator': 2, 'garlic': 3, 'food_jar': 4, 'coffee_mug': 5, 'stapler': 6, 'food_cup': 7, 'notebook': 8, 'cap': 9, 'lemon': 10, 'camera': 11, 'ball': 12, 'dry_battery': 13, 'glue_stick': 14, 'toothpaste': 15, 'greens': 16, 'pliers': 17, 'kleenex': 18, 'scissors': 19, 'shampoo': 20, 'sponge': 21, 'toothbrush': 22, 'marker': 23, 'cereal_box': 24, 'apple': 25, 'potato': 26, 'plate': 27, 'bowl': 28, 'soda_can': 29, 'keyboard': 30, 'flashlight': 31, 'banana': 32, 'instant_noodles': 33, 'food_box': 34, 'pear': 35, 'pitcher': 36, 'comb': 37, 'orange': 38, 'food_can': 39, 'binder': 40, 'mushroom': 41, 'cell_phone': 42, 'hand_towel': 43, 'water_bottle': 44, 'food_bag': 45, 'lightbulb': 46}
mean tensor([0.5547, 0.5310, 0.5190])
std tensor([0.2122, 0.2184, 0.2499])
mean_d tensor([0.7426, 0.2956, 0.4785])
std_d tensor([0.2012, 0.2030, 0.2916])


# 2.2 SynROD dataset


---



1.   SynROD  with label as a trainset



In [ ]:
with open("/content/ROD-synROD/synROD/synARID_50k-split_sync_train1.txt") as f:
    lines = f.read() 
    syn_imgs = lines.split('\n')
syn_imgs.remove('')

with open("/content/ROD-synROD/synROD/synARID_50k-split_sync_test1.txt") as f:
    lines = f.read() 
    syn_imgs_test = lines.split('\n')
# syn_imgs.remove('')

syn_imgs_test.remove('')




syn_all_labels = [syn_imgs[i].split('/')[0] for i in range(len(syn_imgs))]
syn_all_labels=list(set(syn_all_labels))

syn_all_labels_test = [syn_imgs_test[i].split('/')[0] for i in range(len(syn_imgs_test))]
syn_all_labels_test=list(set(syn_all_labels_test))




In [ ]:
SynROD_train = CustomImageDataset(dt = 'SYNROD',annotations_list = syn_imgs,label_map=labelmap,img_dir ='/content/ROD-synROD/synROD',depth_dir ="/content/ROD-synROD/synROD",
                                transform = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(), ]),

                                transform_d = transforms.Compose([
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(), ])
                                
                             
    
)




SynROD_dataloader_train = DataLoader(SynROD_train, batch_size=256,shuffle=True)

mean, std,mean_d,std_d = get_mean_and_std(SynROD_dataloader_train)

print('mean',mean)
print('std',std)
print('mean_d',mean_d)
print('std_d',std_d)



mean tensor([0.3107, 0.2884, 0.2595])
std tensor([0.2352, 0.2268, 0.2336])
mean_d tensor([0.4422, 0.4014, 0.6852])
std_d tensor([0.1382, 0.2507, 0.3628])
